# T-cell chemokine microarrays differential expression analysis

## Index

1. [Experimental design](#Experimental-design)
2. [Loading libraries and dependencies](#Loading-libraries-and-dependencies)
3. [PCA plot of the log transformed raw expression data](#PCA-plot-of-the-log-transformed-raw-expression-data)
4. [Boxplot of log2 intensities for the raw data](#Boxplot-of-log2-intensities-for-the-raw-data)
5. [Background adjustment, normalization, summarization](#Background-adjustment,-normalization,-summarization)
6. [PCA plot of the calibrated, summarized on exon level data](#PCA-plot-of-the-calibrated,-summarized-on-exon-level-data)
7. [tSNE plot of the calibrated, summarized on exon level data](#tSNE-plot-of-the-calibrated,-summarized-on-exon-level-data)
8. [Histogram of the median intensities](#Histogram-of-the-median-intensities)
9. [Probe-based annotation for MTA 1.0](#Probe-based-annotation-for-MTA-1.0)
10. [Contrasts and hypothesis testing using `limma`](#Contrasts-and-hypothesis-testing-using-limma)
10. [Transcript-based annotation for-MTA 1.0](#Transcript-based-annotation-for-MTA-1.0)
11. [Contrasts and hypothesis testing for transcripts using `limma`](#Contrasts-and-hypothesis-testing-for-transcripts-using-limma)
12. [Volcano plot of differentially expressed genes](#Volcano-plot-of-differentially-expressed-genes)
13. [Distrubution of expression values](#Distrubution-of-expression-values)
14. [Heatmap](#Heatmap)
16. [Enrichment analysis](#Enrichment-analysis)
10. [Session info](#Session-info)

## Experimental design

| Chip   | Filename                     | Day post infection | Treatment      | Tissue |
|--------|------------------------------|--------------------|----------------|--------|
| 1      | eberlein_d8_1_Mouse430_2.CEL | 8                  | ex vivo        | spleen |
| 2      | eberlein_d8_2_Mouse430_2.CEL | 8                  | ex vivo        | spleen |
| 3      | eberlein_d8_3_Mouse430_2.CEL | 8                  | ex vivo        | spleen |
| 4      | eberlein_d8_4_Mouse430_2.CEL | 8                  | 3h stimulation | spleen |
| 5      | eberlein_d8_5_Mouse430_2.CEL | 8                  | 3h stimulation | spleen |
| 6      | eberlein_d8_6_Mouse430_2.CEL | 8                  | 3h stimulation | spleen |
| 7      | eberlein_1_1_Mouse430_2.CEL  | 46                 | ex vivo        | spleen |
| 8      | eberlein_1_2_Mouse430_2.CEL  | 46                 | ex vivo        | spleen |
| 9      | eberlein_1_3_Mouse430_2.CEL  | 46                 | ex vivo        | spleen |
| 10     | eberlein_1_4_Mouse430_2.CEL  | 46                 | ex vivo        | spleen |
| 11     | eberlein_1_5_Mouse430_2.CEL  | 46                 | 3h stimulation | spleen |
| 12     | eberlein_1_6_Mouse430_2.CEL  | 46                 | 3h stimulation | spleen |
| 13     | eberlein_1_7_Mouse430_2.CEL  | 46                 | 3h stimulation | spleen |
| 14     | eberlein_1_8_Mouse430_2.CEL  | 46                 | 3h stimulation | spleen |

> The GeneChip™ Mouse Genome 430 2.0 Array is the first and most comprehensive whole mouse genome expression array:
• Complete coverage of the Mouse Expression Set 430 for analysis of over 39,000 transcripts on a single array
• The Power of the Probe Set - offering multiple independent measurements for each transcript delivering the greatest accuracy and reproducibility of any microarray platform
• More information from every experiment for more complete and efficient analysis 

## Loading libraries and dependencies

## Session info

In [ ]:
sessionInfo()